<a href="https://colab.research.google.com/github/MuellerLeonard/Word-embeddings_CNN/blob/master/ft_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fasttext

     |████████████████████████████████| 71kB 2.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3014180 sha256=2b25efd15f95ae1d1d554c5ed89b2e62672c11dabc144b3de6dd390c4dad14d9
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import re
import string
from typing import List
import nltk
from nltk import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

def clean_text(tweets: List[str], lang: str = 'english') -> List[str]:
    """
    performes stemming and other "cleanup"
    :param tweets:
    :param lang:
    :return:
    """

    data: List[str] = tweets
    cleaned_text = []
    stops = set(stopwords.words(lang))

    # no ! and .
    table = str.maketrans(dict.fromkeys(""""#$%&()*+,-/:;<=>?@[\\]^_`{|}~"""))

    for text in data:
        text = re.sub(r'https?://[^\s]*', ' tokenlink ', text)
        text = re.sub(r'\.\.\.', ' tokendotdotdot ', text)
        text = re.sub(r'xD', ' tokenxd ', text)
        text = re.sub(r':\)', ' tokenxbrackethappy ', text)
        text = re.sub(r':\(', ' tokenxbracketsad ', text)
        text = re.sub(r':-\)', ' tokennosehappy ', text)
        text = re.sub(r':-\(', ' tokennosesad ', text)
        text = re.sub(r':D', ' tokenxcheer ', text)
        text = re.sub(r':-S', ' tokens ', text)


        ## Convert words to lower case and split them
        text = text.lower().split()

        text = " ".join(text)

        # Clean the text
        text = re.sub(r"(?<!\.)\.(?!\.)", " ", text)
        text = re.sub(r"[^A-Za-z0-9^,!./'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"-", " - ", text)
        text = re.sub(r"=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)

        ## Remove puncuation
        text = text.translate(table)

        text = text.split()

        ## Remove stop words[^\s][^\s]
        #text = [w for w in text if not w in stops]

        stemmer = SnowballStemmer(lang)
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
        cleaned_text.append(text)

    return cleaned_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
import os
import csv
import math
import fasttext

def load_sentqs_tweets():
  # data/
    if os.path.isfile("sentqs_preprocessed.npz"):
      # data/
        loaded_data = np.load("sentqs_preprocessed.npz")
        return loaded_data['cleaned_tweets'],loaded_data["tweets"],loaded_data['sentiment']
    else:
        hashtags = ['ADBE', 'GOOGL', 'AMZN', 'AAPL', 'ADSK', 'BKNG', 'EXPE', 'INTC', 'MSFT', 'NFLX', 'NVDA', 'PYPL', 'SBUX',
         'TSLA', 'XEL', 'positive', 'bad', 'sad']

        # Loading and preprocessing of tweets
        # only using a small part of available Tweets to not overload the ram
        df = pd.read_csv("/content/drive/My Drive/googleColabFiles/Tweets.csv")
        sentiment = pd.to_numeric(df.iloc[:, -1], errors="raise", downcast="float")
        labels,tweets,sentiment = seperate_tweets(df.iloc[:, 1],hashtags,sentiment)
        cleaned_tweets = clean_text(tweets)

        np.savez_compressed("sentqs_preprocessed.npz",tweets=tweets, cleaned_tweets=cleaned_tweets,sentiment=sentiment)
        return cleaned_tweets,tweets, sentiment

def seperate_tweets(data,hashtags,sentiment):
    print("Seperate Tweets \n")
    labels = []
    tweets = []
    sentiment_new = []
    for t,s in zip(data,sentiment):
        for h in hashtags:
            t = t.lower()
            h = "#" + h.lower()
            if h in t:
                labels.append(h)
                tweets.append(t.replace(h," "))
                sentiment_new.append(s)
                break

    print(sentiment_new)
    return labels,tweets,sentiment_new

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

def text_classification_ft(text, sentiment, train_size=0.8, epochs=25, learningrate=0.5, word_n_grams=2, dim=50, bucket=200000, loss='hs'):
    #sentiment: 2.0 = negative, 1.0 = positive, 0.0 = neutral
    #supervised fastText only for Tweets.csv
    #does not have to be Tokenized
    
    #converting float values of sentiment to string values in a new column
    score = [sentiment.size]
    for x in sentiment:
        if x == 0.0:
          score.append("neutral")  
        elif x == 1.0:
          score.append("positive")
        elif x == 2.0:
          score.append("negative")
        
    score.pop(0)

    print(len(score))
    print(score) 

    #Preprocessing
    #combining individual columns to a table
    texts = [''.join(x) for x in text]
    df = pd.DataFrame(texts)
    dfs = pd.DataFrame(sentiment)
    dfsc = pd.DataFrame(score)
    combined = pd.concat([dfs, df, dfsc], axis=1)
    combined.columns = ['sentiment', 'text', 'score']
    combined.to_csv('tweets.csv', index=False)
    tweets = pd.read_csv("tweets.csv")
    
    print(tweets.head())

    #formatting the data-table so that fastText can use it
    col = ['score', 'text']

    tweets = tweets[col]
    tweets['score']=['__label__'+ s for s in tweets['score']]
    tweets['text']= tweets['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
    tweets.to_csv(r'tweets_updated.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

    print("updated dataframe: ")
    print(tweets.head())
    train_data = math.ceil(len(texts) * train_size)
    test_data = math.floor(len(texts) * (1-train_size))
  
    #split data into head and tail for train and test
    print("splitting data into training and test", train_data, ",", test_data)
    tweets_train = tweets.head(train_data)
    tweets_train.to_csv(r'tweets_train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    tweets_test = tweets.tail(test_data)
    tweets_test.to_csv(r'tweets_test.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

    print("train fastText Model supervised with the training data: ")
    # Hyperparameters can be optimized, using the Standard ones
    modelft = fasttext.train_supervised(input="tweets_train.txt")#, lr=learningrate, epoch=epochs, wordNgrams=word_n_grams, bucket=bucket, dim=dim, loss=loss)
    print("words: ", modelft.words)
    print("labels: ", modelft.labels)
    print("Precision and Recall on test data: ")
    print_results(*modelft.test("tweets_test.txt"))

    print("predict on a sentence and return the probability of the labels: ")
    sentence = "I am not happy with how the issue got handled"
    cleaned_sentence = clean_text(sentence)
    print("sentence: ", cleaned_sentence)
    print(modelft.predict(sentence, k=3))


def main():

    # Load neccessary informations about the dataset
    cleaned_tweets,tweets,sentiment = load_sentqs_tweets()

    # supervised training and testing with fastText on Tweets.csv:
    text_classification_ft(cleaned_tweets, sentiment)

if __name__ == '__main__':
    main()

61529
['negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'neutral', 'positive', 'negative', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'neutral', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'neutral', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'negative', 'neutral', 'neutral', 'negative', 'neutral', 'positiv